# DAY 1 : EDA와 데이터 전처리

In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt


In [ ]:
print('numpy version :', np.__version__)
print('pandas version :', pd.__version__)
print('matplotlib version :', mpl.__version__)

## 1. 데이터 불러오기

In [ ]:
BASE_DIR = './data'
train_path = os.path.join(BASE_DIR, 'train.csv')
test_path = os.path.join(BASE_DIR, 'test.csv')

In [ ]:
data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
target = data['credit']

## 2. 데이터 살펴보기

데이터를 받으면 확인해야할 것

1. 컬럼 별 데이터의 형태 (Type)
2. 데이터 개수 (Shape)
3. 데이터 결측치 (Missing Value) 
4. 데이터 통계값 (Statistics)
5. 데이터의 분포 (Distribution)
6. 데이터의 예외값 (Outlier)

### 2-1. 데이터 대략적으로 이해하기

In [ ]:
data.head()

- `index`: 인덱스
- `gender`: 성별
- `car`: 차량 소유 여부
- `reality`: 부동산 소유 여부
- `child_num`: 자녀 수
- `income_total`: 연간 소득
- `income_type`: 소득 분류
    - ['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']
- `edu_type`: 교육 수준 
    - ['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']
- `family_type`: 결혼 여부 
    - ['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']
- `house_type`: 생활 방식
    - ['Municipal apartment', 'House / apartment', 'With parents', 'Co-op apartment', 'Rented apartment', 'Office apartment']
- `DAYS_BIRTH`: 출생일
    - 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미
- `DAYS_EMPLOYED`: 업무 시작일
    - 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미
    - 양수 값은 고용되지 않은 상태를 의미함
- `FLAG_MOBIL`: 핸드폰 소유 여부
- `work_phone`: 업무용 전화 소유 여부
- `phone`: 전화 소유 여부
- `email`: 이메일 소유 여부
- `occyp_type`: 직업 유형													
- `family_size`: 가족 규모
- `begin_month`: 신용카드 발급 월
    - 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미
- `credit`: 사용자의 신용카드 대금 연체를 기준으로 한 신용도
    - => 낮을 수록 높은 신용의 신용카드 사용자를 의미함

In [ ]:
data.info()

범주형과 수치형을 분류해서 살펴본다.

In [ ]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t=='O'] 
num_columns = [c for c in data.columns if c not in cat_columns]

print(f'Categorical Columns: \n{cat_columns}\n')
print(f'Numeric Columns: \n{num_columns}\n')

In [ ]:
data.describe().T

In [ ]:
data.describe(include='O').T

### 2-2. 데이터 결측치, 분포, 예외치 확인하기

In [ ]:
import missingno as msno

msno.matrix(data, sort='ascending')

In [ ]:
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(1,1)
sns.countplot(data=data, x='gender')
plt.show()

> value_counts를 사용한다면..?

In [ ]:
fig, ax = plt.subplots(1,1)
sns.countplot(data=data, x='credit')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1)
sns.kdeplot(data=data, x='income_total', ax=ax, fill=True)
# sns.histplot(data=data, x='income_total', ax=ax)
# ax.ticklabel_format(style='plain')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1)
sns.kdeplot(data=data, x='income_total', hue='gender', fill=True, ax=ax)
# ax.ticklabel_format(style='plain')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1)
sns.kdeplot(data=data, x='income_total', y='DAYS_EMPLOYED', ax=ax)
# ax.ticklabel_format(style='plain')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1)
sns.scatterplot(data=data, x='income_total', y='DAYS_EMPLOYED', ax=ax)
# ax.ticklabel_format(style='plain')
plt.show()

## 3. 데이터 스케일링

### 3-1. 스케일링을 하는 이유

변수의 크기가 너무 작거나, 너무 큰 경우 해당 변수가 Target 에 미치는 영향력이 제대로 표현되지 않을 수 있습니다.

Sklearn의 대표적인 스케일링 함수로는 특정 변수의 최대, 최소 값으로 조절하는 Min-Max 스케일링과 z-정규화를 이용한 Standard 스케일링이 있습니다.

<div class="alert alert-block alert-danger">
    <b>주의</b> 스케일링은 변수의 분포를 변경하지 않습니다.
</div>

> 주의 > 스케일링은 변수의 분포를 변경하지 않습니다. 

### 3-2. Min-Max Scaling

- Min-Max 스케일링을 하면, 값의 범위가 0 ~ 1 사이로 변경됩니다. <br> 
- 최댓값을 1, 최솟값을 0으로 변경합니다.

수식을 직관적으로 이해해보면, X에 존재하는 어떤 가장 작은 값 x <sub>m</sub>에 대해서 x <sub>m</sub>는 Min(X)의 값과 같습니다. 

따라서 스케일링 후 x<sub>m</sub>은 0이되고, X에 존재하는 어떤 가장 큰 값x <sub>M</sub>은 분모의 식과 같아지므로 1이됩니다.

$$ x - Min(X) \over Max(X) - Min(X) $$<br>
$$X : 데이터\ 셋 $$
$$ x : 데이터\ 샘플 $$ 



직접 구현도 크게 어렵지 않지만, `Sklearn`에서 Min-Max Scaler는 preprocessing 패키지에 있습니다. 

#### Reference

- [Scikit-learn Min-Max Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html?highlight=minmax#sklearn.preprocessing.MinMaxScaler)


In [ ]:
numeric_data = data[num_columns].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(numeric_data)

In [ ]:
scaled_data = scaler.transform(numeric_data)
scaled_data = pd.DataFrame(scaled_data, columns=num_columns)

In [ ]:
data[num_columns].head()

In [ ]:
data[num_columns].describe()

In [ ]:
scaled_data.head()

In [ ]:
scaled_data.describe()

### 3-3. Standard Scaling

데이터를 통계적으로 표준정규분포화 시켜 스케일링을 하는 방식입니다. `z-score 정규화` 라고도 합니다.

데이터의 평균이 0, 표준 편차가 1이 되도록 스케일링 합니다.

$$ z = {{x - \mu} \over {\sigma}} $$
$$ \mu : 데이터의\ 평균, mean(X) $$
$$ \sigma : 데이터의\ 표준편차, std(X)$$
$$ X : 데이터\ 셋 $$
$$ x : 데이터\ 샘플 $$

`Sklearn`에서 Standard Scaler는 preprocessing 패키지에 있습니다.

#### Reference

- [Scikit-learn, Standard Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
scaler.fit(numeric_data)

In [ ]:
scaled_data = scaler.transform(numeric_data)
scaled_data = pd.DataFrame(scaled_data, columns=num_columns)

In [ ]:
data[num_columns].head()

In [ ]:
data[num_columns].describe()

In [ ]:
scaled_data.head()

In [ ]:
scaled_data.describe()

기존 데이터에서는 변수별로 서로 다른 평균과 표준 편차 값을 가지고 있었습니다.

Standard 스케일링된 데이터를 살펴보면, 평균이 0 표준편차가 1이 되었음을 확인할 수 있습니다.

### 3-4. Log Transformation

변수의 범위를 조정하는 Min-Max Scaling과 Standard Scaling을 확인했습니다.<br>
이번에는 치우친 분포(skew)의 형태를 보정해주는 방법 중 하나인 Log Transformation에 대해 배우겠습니다.

로그 변환은 어떤 변수의 범위가 양수인 경우 사용할 수 있으며, 각 변수에 대해 자연 로그를 취하는 연산 입니다.

해당 연산을 적용하면, 다음과 같은 왼쪽으로 치우친 분포(왼쪽)를 좋은 형태(오른쪽)의 정규 분포의 형태로 만들어줄 수 있습니다.

이는 왼쪽 그림에서 오른쪽으로 치우친 큰 값들을 작게(지수로) 표현함으로써 분포를 왼쪽으로 당기는 효과를 줄 수 있기 때문입니다.

<img src='https://miro.medium.com/max/1620/1*O2R4nH0nR6d3bOxaYU10lg.png'>

#### ref
- [Log Transformation](https://medium.com/@kyawsawhtoon/log-transformation-purpose-and-interpretation-9444b4b049c9)

In [ ]:
# 로그 변환 전
sns.histplot(data=data, x='income_total', bins=20)

In [ ]:
# 로그 변환 후
scaled_data['log_income_total'] = np.log(data['income_total'])
scaled_data['log_income_total'].hist(bins=20)

### 3-5. Box-Cox, Yeo-Johnson, Quantile Transformation

여러가지 변환이 있습니다. 

세가지 변환 모두 치우친 분포의 데이터를 정규 분포로 보정해주는 효과가 있으며,

Box-Cox 변환의 경우 Log 변환과 함께 양수에만 적용할 수 있는 변환이고, 

Yeo-Johnsong 변환은 Box-Cox 변환의 제약인 음수를 가진 변수에서도 변환을 할 수 있는 변환입니다.

Quantile 변환은 가장 자주 발생하는 값(the most frequent values.) 주위로 분포를 조정하며, 이상치의 영향을 감소시켜주는 특징도 있습니다.

- Box-Cox 변환은 Lambda 값에 따라 변환이 달라지게 되는데, Lambda 값이 0일 경우 Log 변환과 동일합니다.

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_map_data_to_normal_001.png'>

#### Reference
- [Scikit-learn PowerTransformer, Box-Cox, Yeo-Johnson Transformation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer)
- [Scikit-learn Quantile Transformation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html#sklearn.preprocessing.QuantileTransformer)
- [Scikit-learn Map Data To Normal Dist](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_map_data_to_normal.html#sphx-glr-auto-examples-preprocessing-plot-map-data-to-normal-py)

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Box-Cox 변환 후
trans = PowerTransformer(method='box-cox')

scaled_data['box_cox_income_total'] = trans.fit_transform(scaled_data['income_total'].values.reshape(-1, 1))
scaled_data['box_cox_income_total'].hist(bins=20)

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Yeo-Johnson 변환 후
trans = PowerTransformer(method='yeo-johnson')

scaled_data['yeo_johnson_income_total'] = trans.fit_transform(scaled_data['income_total'].values.reshape(-1, 1))
scaled_data['yeo_johnson_income_total'].hist(bins=20)

In [ ]:
from sklearn.preprocessing import QuantileTransformer

# Quantile 변환 후
trans = QuantileTransformer(output_distribution='normal')

scaled_data['quantile_income_total'] = trans.fit_transform(scaled_data['income_total'].values.reshape(-1, 1))
scaled_data['quantile_income_total'].hist(bins=20)

## 4. Missing Values


### 4-1. 대표 값을 사용한 결측치 처리

정형 데이터를 다루다보면, 값이 NaN(Not a Number or Null)으로 되어있는 경우가 있습니다. 이러한 값을 결측치라 하며, 

가장 간단한 방법으로 평균이나 중간값 또는 최빈값 같은 변수의 대표값을 사용할 수 있습니다.

결측치를 확인하는 방법으로 missingno 라이브러리의 `matrix()` 메소드를 사용하거나 Pandas의 `isna() + sum()` 메소드를 사용할 수 있습니다.

<div class="alert alert-block alert-danger">
    <b>주의</b> 결측치 처리는 가능하면 스케일링 작업 이전에 해주는 것이 좋습니다. 예를 들어 특정 컬럼의 결측치를 평균 값으로 대치하려고할 때 Standard Scaling을 진행하게되면 해당 컬럼의 결측치는 모두 0으로 대치될 수 있기 때문입니다. 
</div>


In [ ]:
msno.matrix(data)

In [ ]:
na_list = pd.isna(data).sum()
na_list[na_list > 0]

### 4-2. 평균(mean)

확률 이론과 통계 관점에서 (모)평균 또는 기댓값은 중심에 대한 경향성을 알 수 있는 척도입니다.

일반적으로 평균이라고 부르는 것으로 산술 평균이라고 하고, 이 평균은 표본 평균이라고도 합니다.

평균은 모든 관측치의 값을 모두 반영하므로 `지나치게 작거나 큰 값(이상치)들의 영향을 많이` 받게 됩니다.

평균은 모든 샘플의 값을 더하고, 샘플의 개수로 나누어 계산할 수 있습니다.

$$ E(x) = {\sum x \over n}$$

In [ ]:
mean_df = data.copy()

In [ ]:
mean_df

In [ ]:
for c in num_columns:
    mean_df.loc[pd.isna(data[c]), c] = data[c].mean()

In [ ]:
pd.isna(mean_df[num_columns]).sum()

### 4-3. Median(중간값) 

중간값은 데이터 샘플을 개수에 대해서 절반으로 나누는 위치의 값을 말합니다.

데이터 샘플의 수가 짝수개일 때에는 중간에 위치한 두 값의 평균을 사용합니다.

중간값은 모든 관측치의 값을 모두 반영하지 않으므로 `지나치게 작거나 큰 값(이상치)들의 영향을 덜` 받습니다.<br>
중간값은 샘플을 값에 대해 정렬하고, 중앙에 위치한 값으로 구할 수 있습니다.

In [ ]:
median_df = data.copy()

for c in num_columns:
    median_df.loc[pd.isna(data[c]), c] = data[c].median()

In [ ]:
pd.isna(median_df[num_columns]).sum().sum()

### 4-4. Iterative Impute (R 언어의 MICE 패키지)

Round robin 방식으로 반복하여 결측값을 회귀하는 방식으로 결측치를 처리합니다. 

결측값을 회귀하는 방식으로 처리하기 때문에 수치형 변수에만 적용할 수 있습니다.

1. 각 결측치를 해당 변수의 평균으로 채워넣는다. 
2. 대체할 변수의 결측치는 제외한 상태로 해당 변수의 결측치를 회귀모델을 이용하여 예측한다.
3. 다른 변수에서도 해당 방식을 반복한다.
4. 모든 변수에 대해 반복 후 해당 이터레이션에서 맨 처음에 할당했던 값과의 차이를 계산한다.
5. 해당 값의 차이가 0이 될 때(수렴)까지 반복한다.

#### Reference
- [Scikit-Learn, Iterative Imputer (MICE)](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html?highlight=mice)
- [MICE 알고리즘 설명](https://ichi.pro/ko/deiteo-seteueseo-gyeol-cheuggabs-eul-daechihaneun-mice-algolijeum-217004654686142)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
impute_df = data.copy()

In [ ]:
imp_mean = IterativeImputer(random_state=0)
impute_df[num_columns] = imp_mean.fit_transform(impute_df[num_columns])

In [ ]:
pd.isna(impute_df[num_columns]).sum().sum()

### 4-5. Mode(최빈값)

최빈값은 범주형 변수에서 가장 자주 등장하는 값을 말합니다.

In [ ]:
mode_df = data.copy()

In [ ]:
for c in cat_columns:
    mode_df.loc[pd.isna(data[c]), c] = data[c].mode()[0]

In [ ]:
pd.isna(mode_df[cat_columns]).sum().sum()

## 5. Categorical Variable to Numeric Variable 

이번에는 범주형 변수를 수치형 변수로 나타내는 방법에 대해 알아보겠습니다. 
여기에서 범주형 변수란, 차의 등급을 나타내는 [소형, 중형, 대형] 처럼 표현되는 변수를 말합니다. 

범주형 변수는 주로 데이터 상에서 문자열로 표현되는 경우가 많으며, 문자와 숫자가 매핑되는 형태로 표현되기도 합니다.

### 5-1. Label Encoding
라벨 인코딩은 n개의 범주형 데이터를 0~n-1 의 연속적인 수치 데이터로 표현합니다.<br>
예를 들어, 차의 등급 변수를 라벨 인코딩으로 변환하면 다음과 같이 표현할 수 있습니다.<br>

- 소형 : 0 
- 중형 : 1 
- 대형 : 2 

라벨 인코딩은 간단한 방법이지만, '소형'과 '중형'이라는 범주형 데이터가 가지고 있는 차이가 0과 1의 수치적인 차이라는 의미가 아님을 주의하셔야 합니다. 

Label Encoding도 Sklearn의 preprocessing 패키지에 있습니다.

#### Reference
- [Scikit-Learn Label Encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html?highlight=label%20encoder#sklearn.preprocessing.LabelEncoder)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
le.fit(data['family_type'])

In [ ]:
# classes_ 속성에 있는 순서(index)대로 라벨 번호가 부여됩니다.
le.classes_

In [ ]:
label_encoded = le.transform(data['family_type'])

In [ ]:
result = pd.DataFrame(data = np.concatenate([data['family_type'].values.reshape((-1,1)), label_encoded.reshape((-1, 1))], axis=1), 
                      columns=['label', 'label_encoded'])
result.head(20)

### 5-2. One-hot Encoding

원핫 인코딩은 n개의 범주형 데이터를 n개의 비트(0,1) 벡터로 표현합니다. 

예를 들어, 위에서 언급한 소형, 중형, 대형으로 이루어진 범주형 변수를 원핫 인코딩을 통해 변환하면 다음과 같이 표현할 수 있습니다.

- 소형 : `[1, 0, 0]` 
- 중형 : `[0, 1, 0]` 
- 대형 : `[0, 0, 1]` 

원핫 인코딩으로 범주형 데이터를 나타내게되면, 서로 다른 범주에 대해서는 벡터 내적을 취했을 때 내적 값이 0이 나오게 됩니다. 

이는 서로 다른 범주 데이터는 독립적인 관계라는 것을 표현할 수 있게 됩니다.

One-hot Encoding은 Sklearn의 preprocessing 패키지에 있습니다.

#### Reference
- [Scikit-Learn One-Hot Encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [ ]:
ohe.fit(data[['family_type']])

In [ ]:
ohe.categories_

In [ ]:
one_hot_encoded = ohe.transform(data[['family_type']])

In [ ]:
one_hot_encoded

In [ ]:
columns = np.concatenate([np.array(['family_type']) , ohe.categories_[0]])

result = pd.DataFrame(data=
                      np.concatenate([data[['family_type']].values, 
                                      one_hot_encoded.reshape((-1, ohe.categories_[0].shape[0]))],                                             
                                     axis=1), 
                      columns=columns)
result.head(10)

## 6. Test

In [ ]:
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv(train_path)
data = data.drop(['credit'], axis=1)
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t=='O'] 
num_columns = [c for c in data.columns if c not in cat_columns]

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)

decision_tree = decision_tree.fit(data[num_columns], target)

In [ ]:
result = decision_tree.predict_proba(test_data[num_columns])

In [ ]:
answer = pd.read_csv(os.path.join(BASE_DIR, 'sample_submission.csv'))

In [ ]:
answer['0'] = result[:, 0]
answer['1'] = result[:, 1]
answer['2'] = result[:, 2]

In [ ]:
answer.to_csv('test.csv', index=False)

In [ ]:
answer